In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
details = []
casts = []
for root, dirs, files in os.walk('data/tmdb_divided'):
    for file in files:
        if file.endswith('.csv'):
            if 'detail' in file:
                details.append(os.path.join(root, file))
            else:
                casts.append(os.path.join(root, file))
details.sort()
casts.sort()

In [3]:
detail_df = pd.DataFrame()
for detail in details:
    df = pd.read_csv(detail).drop('Unnamed: 0', axis=1)
    detail_df=pd.concat([detail_df, df], ignore_index=True)

/var/folders/ny/4mt1_15d103gx2y1gsxzf4t00000gn/T/ipykernel_36594/2079162657.py:3: DtypeWarning: Columns (1,23,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(detail).drop('Unnamed: 0', axis=1)


In [4]:
cast_df = pd.DataFrame()
for cast in casts:
    print(cast)
    df = pd.read_csv(cast).drop('Unnamed: 0', axis=1)
    cast_df=pd.concat([cast_df, df], ignore_index=True)

data/tmdb_divided/tmdb_movies_casts_0.csv
data/tmdb_divided/tmdb_movies_casts_1.csv
data/tmdb_divided/tmdb_movies_casts_2.csv
data/tmdb_divided/tmdb_movies_casts_3.csv
data/tmdb_divided/tmdb_movies_casts_4.csv
data/tmdb_divided/tmdb_movies_casts_5.csv
data/tmdb_divided/tmdb_movies_casts_6.csv
data/tmdb_divided/tmdb_movies_casts_7.csv


In [5]:
pd.set_option('display.max_rows', None)  # or set a high number like 500
pd.set_option('display.max_columns', None)  # or set a high number if you have many columns

detail_df[detail_df['id'] == 89280]

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,success,status_code,status_message
129223,False,/1Tkjtb9Hl8TdrmXUGbYq1h2ota8.jpg,NaN,0.0,Drama,NaN,89280.0,tt0273850,en,My Brother Tom,"A teenage girl, Jessica, befriends a teenage b...",5.725,/As8XfSgzm2ZUZItwuGlvLepBJdg.jpg,Film4 Productions,United Kingdom,2001-11-16,0.0,111.0,English,Released,Love and tragedy is just a short walk from hom...,My Brother Tom,False,6.1,24.0,NaN,NaN,NaN


In [7]:
detail_id = detail_df['id']
cast_id = cast_df['id']

In [8]:

# Convert lists to sets
detail_set = set(detail_id)
cast_set = set(cast_id)

# Find unique values in each list
unique_in_list1 = detail_set - cast_set  # Values unique to list1
unique_in_list2 = cast_set - detail_set  # Values unique to list2

# Alternatively, to find values that are unique in each list in one step
unique_in_both = detail_set.symmetric_difference(cast_set)

In [9]:
custom_tmdb_df = pd.merge(left=detail_df, right=cast_df, on='id', how='inner')

In [10]:
filtered_df = custom_tmdb_df.dropna(subset='overview')

In [11]:
filtered_df2 = filtered_df[filtered_df['status'].str.lower() == 'released']

In [16]:
filtered_df2 = filtered_df2[['id', 'title', 'budget', 'revenue', 'genres','imdb_id',
       'overview', 'tagline', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime',
       'vote_average', 'vote_count', 'casts', 'director', 'director_of_photography',
       'producer', 'writer', 'music_composer']]

In [17]:
filtered_df2.to_csv('data/custom_tmdb.csv')

In [23]:
no_genres = filtered_df2[filtered_df2['genres'].isna()]

In [28]:
filtered_df3 = filtered_df2.dropna(subset='genres')

In [31]:
no_cast = filtered_df3[filtered_df3['casts'].isna()]
no_director = filtered_df3[filtered_df3['director'].isna()]

In [33]:
no_cast.describe()

,id,budget,revenue,popularity,runtime,vote_average,vote_count
count,1.079010e+05,1.079010e+05,1.079010e+05,107901.000000,107901.000000,107901.000000,107901.000000
mean,7.285886e+05,7.151126e+03,6.185404e+03,0.802371,36.311165,1.541785,1.096765
std,3.328223e+05,6.253012e+05,1.553162e+06,0.901308,68.822105,2.849798,9.175618
min,8.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,4.551870e+05,0.000000e+00,0.000000e+00,0.600000,6.000000,0.000000,0.000000
50%,7.524110e+05,0.000000e+00,0.000000e+00,0.600000,18.000000,0.000000,0.000000
75%,1.021182e+06,0.000000e+00,0.000000e+00,0.662000,60.000000,1.000000,1.000000
max,1.257774e+06,1.500000e+08,5.000000e+08,131.810000,14400.000000,10.000000,1561.000000


In [38]:
filtered_df4 = filtered_df3.dropna(subset='director')

In [40]:
filtered_df4['vote_count'].quantile([0.8, 0.9, 0.95, 0.99, 0.995])

0.800       7.0
0.900      21.0
0.950      65.0
0.990     766.0
0.995    1807.0
Name: vote_count, dtype: float64

In [65]:
filtered_df4[filtered_df4['vote_count']>=1042].shape[0], filtered_df4[filtered_df4['vote_count']>=890].shape[0], filtered_df4[filtered_df4['vote_count']>=766].shape[0]

(3984, 4481, 4976)

In [56]:
filtered_df4 = filtered_df4.dropna(subset='vote_count')

In [62]:
filtered_df4['vote_count'].quantile([0.8, 0.9, 0.95, 0.99, 0.991, 0.992, 0.995])

0.800       7.0
0.900      21.0
0.950      65.0
0.990     766.0
0.991     890.0
0.992    1042.0
0.995    1807.0
Name: vote_count, dtype: float64

In [66]:
threshold = filtered_df4['vote_count'].quantile(0.99)
final_df = filtered_df4[filtered_df4['vote_count']>=threshold]
final_df.to_csv('data/filtered_custom_tmdb.csv')

In [67]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4976 entries, 2 to 904182
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       4976 non-null   float64
 1   title                    4976 non-null   object 
 2   budget                   4976 non-null   float64
 3   revenue                  4976 non-null   float64
 4   genres                   4976 non-null   object 
 5   imdb_id                  4975 non-null   object 
 6   overview                 4976 non-null   object 
 7   tagline                  4563 non-null   object 
 8   popularity               4976 non-null   float64
 9   poster_path              4975 non-null   object 
 10  production_companies     4970 non-null   object 
 11  production_countries     4974 non-null   object 
 12  release_date             4976 non-null   object 
 13  runtime                  4976 non-null   float64
 14  vote_average             49

In [68]:
final_df[final_df['poster_path'].isna()]

,id,title,budget,revenue,genres,imdb_id,overview,tagline,popularity,poster_path,production_companies,production_countries,release_date,runtime,vote_average,vote_count,casts,director,director_of_photography,producer,writer,music_composer
596111,836466.0,Return,0.0,0.0,Animation,NaN,A single man has worked most of his life in a ...,NaN,4.981,NaN,NaN,NaN,2020-06-10,12.0,6.441,918.0,NaN,['Jonathan Delgado Herrera'],[],[],[],NaN
